# Amazon SageMaker
### <font color='blue'>Deploy de Modelo Para Previsão de Doenças Usando Regsitros Médicos Eletrônicos</font>

## Parte 3 - Segunda Versão do Modelo e Criação do Endpoint

## Imports 

https://pypi.org/project/boto/

https://sagemaker.readthedocs.io/en/stable/

In [ ]:
# Imports
import os
import json
import sagemaker
import boto3
import numpy as np
import pandas as pd
from sagemaker.serializers import CSVSerializer
from sagemaker.inputs import TrainingInput
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role

In [ ]:
sagemaker.__version__

## Carrega os Dados

In [ ]:
# Obtém a sessão do SageMaker
session = boto3.Session()

In [ ]:
s3 = session.resource('s3')

In [ ]:
s3

In [ ]:
from sagemaker import get_execution_role
role = get_execution_role()
print(role)

In [ ]:
# Altere para o nome do seu bucket
s3_bucket = 'dsa-deploy-app'
prefix = 'dados'

In [ ]:
raiz = 's3://{}/{}/'.format(s3_bucket, prefix)
print(raiz)

In [ ]:
dados_treino = TrainingInput(s3_data = raiz + 'treino.csv', content_type = 'csv')
dados_teste = TrainingInput(s3_data = raiz + 'teste.csv', content_type = 'csv')

In [ ]:
print(json.dumps(dados_treino.__dict__, indent = 2))

In [ ]:
print(json.dumps(dados_teste.__dict__, indent = 2))

## Construção e Treinamento do Modelo

In [ ]:
# Criação do Container
# https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html
container_uri = sagemaker.image_uris.retrieve(region = session.region_name, 
                                              framework = 'xgboost', 
                                              version = '1.0-1', 
                                              image_scope = 'training')

In [ ]:
# Argumentos do estimador
sagemaker_execution_role = role
sagemaker_session = sagemaker.Session()

In [ ]:
# Criação do Estimador
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
xgb = sagemaker.estimator.Estimator(image_uri = container_uri,
                                    role = sagemaker_execution_role, 
                                    instance_count = 2, 
                                    instance_type = 'ml.m5.large',
                                    output_path = 's3://{}/artefatos'.format(s3_bucket),
                                    sagemaker_session = sagemaker_session,
                                    base_job_name = 'classifier')

In [ ]:
# Definição dos Hiperparâmetros
# https://docs.aws.amazon.com/pt_br/sagemaker/latest/dg/xgboost_hyperparameters.html
xgb.set_hyperparameters(objective = 'binary:logistic', num_round = 100)

In [ ]:
# Treinamento
xgb.fit({'train': dados_treino, 'validation': dados_teste})

## Gerando o Endpoint a Partir do Modelo

In [ ]:
# Deploy do modelo treinado criando o endpoint
# https://docs.aws.amazon.com/pt_br/sagemaker/latest/dg/xgboost.html
xgb_predictor = xgb.deploy(initial_instance_count = 2, instance_type = 'ml.m5.large')

## Previsões a Partir do Endpoint

In [ ]:
csv_serializer = CSVSerializer()

In [ ]:
predictor = Predictor(endpoint_name = xgb_predictor.endpoint_name, serializer = csv_serializer)

In [ ]:
df_teste = pd.read_csv(raiz + 'teste.csv', names = ['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])

In [ ]:
df_teste.head()

In [ ]:
X = df_teste.sample(1)
X

In [ ]:
X = X.values[0]
X[1:]

In [ ]:
paciente = X[1:]
paciente

In [ ]:
# Faz a previsão de um paciente
predicted_class_prob = predictor.predict(paciente).decode('utf-8')
if float(predicted_class_prob) < 0.5:
    print('Previsão = Não Diabético')
else:
    print('Previsão = Diabético')
print()

## Avaliando o Modelo

In [ ]:
# Previsão de todos os pacientes no dataset de teste
predictions = []
expected = []
correct = 0
for row in df_teste.values:
    expected_class = row[0]
    payload = row[1:]
    predicted_class_prob = predictor.predict(payload).decode('utf-8')
    predicted_class = 1
    if float(predicted_class_prob) < 0.5:
        predicted_class = 0  
    if predicted_class == expected_class:
        correct += 1
    predictions.append(predicted_class)
    expected.append(expected_class)

In [ ]:
print('Acurácia = {:.2f}%'.format(correct/len(predictions) * 100))

#### Confusion Matrix

In [ ]:
expected = pd.Series(np.array(expected))
predictions = pd.Series(np.array(predictions))
pd.crosstab(expected, predictions, rownames = ['Actual'], colnames = ['Predicted'], margins = True)